# VLB Stress Relaxation: NLSQ → NUTS

**Liquid foam — stress relaxation modulus G(t)**

## Learning Objectives

- Fit relaxation modulus G(t) = G₀ exp(-k_d t)
- Extract relaxation time t_R = 1/k_d from experimental data
- Compare NLSQ and NUTS estimates

## Data Source

Liquid foam stress relaxation from pyRheo demo datasets (1692 raw points, subsampled).

## Estimated Runtime

- Fast demo (1 chain): ~1 min
- Full run (4 chains): ~3 min

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax openpyxl
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import warnings
import time
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models import VLBLocal

jax, jnp = safe_import_jax()
verify_float64()
warnings.filterwarnings('ignore', category=FutureWarning)

# Tutorial utilities
import sys
sys.path.insert(0, str(Path('..').resolve()))
from utils.vlb_tutorial_utils import (
    get_bayesian_config, print_convergence, print_parameter_table,
    plot_trace_and_forest, get_output_dir, save_results, save_figure,
    load_foam_relaxation,
)

print(f"JAX version: {jax.__version__}")
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

## 2. Load Experimental Data

In [ ]:
time_data, G_data = load_foam_relaxation(max_points=200)

print(f'Data points: {len(time_data)}')
print(f'Time range: {time_data.min():.3f} - {time_data.max():.1f} s')
print(f'G(t) range: {G_data.min():.2f} - {G_data.max():.1f} Pa')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(time_data, G_data, 'ko', markersize=3)
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('G(t) [Pa]')
ax1.set_title('Stress Relaxation')
ax1.grid(True, alpha=0.3)

ax2.semilogy(time_data, G_data, 'ko', markersize=3)
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('G(t) [Pa]')
ax2.set_title('G(t) on log scale (straight = single exponential)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 3. VLB Forward Model

Maxwell stress relaxation modulus:

$$G(t) = G_0 \exp(-k_d \cdot t) = G_0 \exp(-t/t_R)$$

- **G₀** sets the initial modulus at $t=0$
- **k_d = 1/t_R** sets the relaxation rate
- A straight line on semilog(G) vs t confirms single-exponential

## 4. Step 1: NLSQ Point Estimation

In [ ]:
model = VLBLocal()

t0 = time.time()
model.fit(time_data, G_data, test_mode='relaxation')
t_nlsq = time.time() - t0

print(f'NLSQ fit time: {t_nlsq:.2f} s')
print(f'G0  = {model.G0:.2f} Pa')
print(f'k_d = {model.k_d:.4f} 1/s')
print(f't_R = {1/model.k_d:.4f} s')

# Fit quality
time_fine = np.linspace(time_data.min(), time_data.max(), 300)
G_pred_fine = model.predict(time_fine, test_mode='relaxation')
G_pred = model.predict(time_data, test_mode='relaxation')
ss_res = np.sum((G_data - np.array(G_pred))**2)
ss_tot = np.sum((G_data - np.mean(G_data))**2)
r2 = 1 - ss_res / ss_tot
print(f'R-squared = {r2:.6f}')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.semilogy(time_data, G_data, 'ko', markersize=3, label='Data')
ax1.semilogy(time_fine, G_pred_fine, 'r-', lw=2, label='VLB fit')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('G(t) [Pa]')
ax1.set_title('Relaxation Modulus Fit')
ax1.legend()
ax1.grid(True, alpha=0.3)

residuals = (G_data - np.array(G_pred)) / G_data * 100
ax2.plot(time_data, residuals, 'ko-', markersize=3)
ax2.axhline(0, color='gray', ls='--')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Relative residual [%]')
ax2.set_title('Residuals')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. Step 2: Bayesian Inference (NUTS)

In [ ]:
config = get_bayesian_config()
initial_values = {'G0': float(model.G0), 'k_d': float(model.k_d)}
print(f'Config: {config}')
print(f'Warm-start: {initial_values}')

t0 = time.time()
result = model.fit_bayesian(
    time_data, G_data, test_mode='relaxation',
    initial_values=initial_values,
    seed=42,
    **config,
)
t_bayes = time.time() - t0
print(f'\nBayesian inference time: {t_bayes:.1f} s')

## 5. Convergence Diagnostics

In [ ]:
param_names = ["G0", "k_d"]
converged = print_convergence(result, param_names)

In [ ]:
display_arviz_diagnostics(result, param_names, fast_mode=os.environ.get("FAST_MODE", "1") == "1")

## 6. Posterior Summary

In [ ]:
posterior = result.posterior_samples
nlsq_vals = {"G0": model.G0, "k_d": model.k_d}
print_parameter_table(["G0", "k_d"], nlsq_vals, posterior)

## 7. Posterior Predictive Check

In [ ]:
posterior = result.posterior_samples
n_draws = min(200, len(posterior['G0']))

G_samples = []
for i in range(n_draws):
    model.parameters.set_value('G0', float(posterior['G0'][i]))
    model.parameters.set_value('k_d', float(posterior['k_d'][i]))
    G_samples.append(np.array(
        model.predict(time_fine, test_mode='relaxation')
    ))

G_arr = np.array(G_samples)

fig, ax = plt.subplots(figsize=(9, 6))
ax.fill_between(time_fine,
    np.percentile(G_arr, 2.5, axis=0),
    np.percentile(G_arr, 97.5, axis=0),
    alpha=0.3, color='C0', label='95% CI')
ax.semilogy(time_fine, np.median(G_arr, axis=0),
            'C0-', lw=2, label='Posterior median')
ax.semilogy(time_data, G_data, 'ko', markersize=3, label='Data')
ax.set_xlabel('Time [s]')
ax.set_ylabel('G(t) [Pa]')
ax.set_title('Posterior Predictive Check')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 8. Save Results

In [ ]:
output_dir = get_output_dir("stress_relaxation")
save_results(output_dir, model, result, ["G0", "k_d"], None)
print("Done.")

## Key Takeaways

1. **Single-exponential relaxation** matches Maxwell/VLB well
2. **G₀** is the initial modulus, **k_d** is the relaxation rate
3. **Both parameters are identifiable** from relaxation data
4. **Multi-exponential decay** would require VLBMultiNetwork

## Next

- **Notebook 14**: Startup shear NLSQ → NUTS